In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
from pprint import pprint
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

#eventdir = '../input/baseball-events-from-retrosheetorg/'
processed_data_dir = '../input/baseball-events-from-retrosheetorg/processed/processed/'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

firstyear = 1915
finalyear = 1916
league_list = ['A', 'N']
#league = 'N

new_columns = ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b', 'runner_3b']

final_column_order = ['gameID',
 'event_in_game',
 'inning',
 'visitor_or_home',
  'vis_score',
 'home_score',
 'outs',
 'runner_1b',
 'runner_2b',
 'runner_3b',
 'batterID',
 'batter_hand',
 'pitch_count',
 'pitch_sequence',
 'play_orig',
 'pitcherID',
 'pitcher_hand',
 'field_C_playerID',
 'field_1B_playerID',
 'field_2B_playerID',
 'field_3B_playerID',
 'field_SS_playerID',
 'field_LF_playerID',
 'field_CF_playerID',
 'field_RF_playerID',
 'bat_DH_playerID',
 'pinch_runner_playerID',
 'pinch_runner_replaced_playerID',
 'comments',
 'badj',
 'padj',
 'theyear',
 'visteam',
 'hometeam',
 'baserunning',
 'play_description',
 'theplay',
 'baserunning_cleaned']

early_end_game_list = ['PHI191507210', 'CHN191607180', 'NY1191609080', 'NY1191609182']

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")
g = 0

print('ok')
#games_df


ok


In [2]:
s = time.time()
print('reading input data...')
print('reading rosters...')
rosters_df = pandas.read_csv(processed_data_dir+'rosters.csv', low_memory=False, index_col='rownumber')

print('\tkeeping rosters in selected year(s)...')

print('reading games...')
games_df = pandas.read_csv(processed_data_dir+'games.csv', low_memory=False, index_col='gameID', encoding='utf-8', converters={'visitor_linescore': lambda x: str(x), 'home_linescore': lambda x: str(x)})

print('\tfixing linescores...')
# games_df.loc[games_df['visitor_linescore'] == '"nan"', 'visitor_linescore'] = np.nan
# games_df.loc[games_df['home_linescore'] == '"nan"', 'home_linescore'] = np.nan
# games_df.loc[:, 'visitor_linescore'] = games_df['visitor_linescore'].apply(lambda x: str(str(x)[1:-1]))
# games_df.loc[:, 'home_linescore'] = games_df['home_linescore'].apply(lambda x: str(str(x)[1:-1]))
games_df = games_df.assign(has_linescores = True)
games_df.loc[(games_df['visitor_linescore'].isnull()) | (games_df['home_linescore'].isnull()), 'has_linescores'] = False

print('\tchecking for DH...')
has_dh = True
if (finalyear < 1973):
    has_dh = False

print('\tgetting fielder columns...')
fielder_columns = ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID']
fielder_columns += ['field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID'] 
if (has_dh):
    fielder_columns += ['bat_DH_playerID']


print('selecting games and roster rows {0:.0f} tot en met {1:.0f}...'.format(firstyear, finalyear))
games_df.loc[:, 'date'] = pandas.to_datetime(games_df['date'], format='%Y%m%d', errors='coerce')
games_df = games_df[games_df['date'].apply(lambda x: ((x.year >= firstyear) and (x.year <= finalyear)))]
rosters_df = rosters_df[(rosters_df['theyear'] >= firstyear) & (rosters_df['theyear'] <= finalyear)]


if ((firstyear >= 1915) & (finalyear <= 1930)):
    events_df = pandas.DataFrame()
    #### NOTE: only add dirs to eventdirs if they are in the desired firstyear-to-finalyear range!
    eventdirs = sorted([processed_data_dir+x for x in os.listdir(processed_data_dir) if (x.find('.') == -1) & (x[4].upper() in league_list)])

    eventfile_list = []
    eventfile_list += [processed_data_dir+'/1915f/'+x for x in os.listdir(processed_data_dir+'/1915f/') if 'events' in x]
    for this_event_dir in eventdirs:
        eventfile_list += [this_event_dir+'/'+x for x in os.listdir(this_event_dir) if ('events' in x)]
    
#     pprint(eventfile_list)
#     print('=====')
    for thisfile in eventfile_list:
#         print(thisfile)
#         print('---------')
        events_i_df = pandas.read_csv(thisfile, encoding='utf-8', index_col='eventID', low_memory=False)#, dtype={'pitch_count': 'str'})
        events_df = pandas.concat((events_df, events_i_df), axis=0)
else:
    events_df = pandas.read_csv(processed_data_dir+'/events_2020_2020.csv', encoding='utf-8', index_col='eventID')#, dtype={'pitch_count': 'str'}

events_df = events_df.reset_index(drop=True)
events_df.index.name = 'eventID'
print('\tfixing pitch counts so they to be properly zero-padded when needed...')
try:
    events_df.loc[:, 'pitch_count'] = events_df['pitch_count'].apply(lambda x: '{0:02d}'.format(x))
except ValueError:
    print("\t\tCAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...")
    events_df.loc[:, 'pitch_count'] = pandas.to_numeric(events_df['pitch_count'], errors='coerce')

print('splitting play into play description and baserunning...')
events_df = events_df.rename(columns={'theplay': 'play_orig'})
events_df = events_df.assign(baserunning = events_df[(events_df['play_orig'].apply(lambda x: '.' in x))]['play_orig'].apply(lambda x: x[x.find('.')+1:]))
events_df = events_df.assign(play_pre_baserunning = events_df['play_orig'].apply(lambda x: x.split('.')[0]))
events_df = events_df.assign(play_description = events_df[(events_df['play_pre_baserunning'].apply(lambda x: '/' in x))]['play_pre_baserunning'].apply(lambda x: x[x.find('/')+1:]))
events_df = events_df.assign(theplay = events_df['play_pre_baserunning'].apply(lambda x: x.split('/')[0]))
events_df = events_df.drop('play_pre_baserunning', axis=1)

print('cleaning baserunning info...')
events_df = events_df.assign(baserunning_cleaned = events_df['baserunning'].dropna().apply(lambda x: x.replace('B','0').replace('H', '4')))


print('creating a new dataframe to hold what we calculate here...')
new_things_df = pandas.DataFrame(data=None, columns=new_columns, index=events_df.index)

# print('backing up...')
# games_df_bk = games_df
# rosters_df_bk = rosters_df
# events_df_bk = events_df
# new_things_df_bk = new_things_df

e = time.time()
g += e-s
print('\n')
print('Read {0:,.0f} games and {1:,.0f} roster entries and {2:,.0f} events in {3:.1f} seconds!'.format(len(games_df), len(rosters_df), len(events_df) ,e-s))
# crazy game: 
#games_df.loc['BRO191508312'][['visitor_score', 'home_score']]

#events_df.head(1).T

# #games_df.loc['BLF191504100'][['visitor', 'visitor_score', 'visitor_linescore', 'home', 'home_score', 'home_linescore']]
#games_df[['visitor_linescore', 'home_linescore']][games_df['visitor_linescore'] == '000000001']


reading input data...
reading rosters...
	keeping rosters in selected year(s)...
reading games...
	fixing linescores...
	checking for DH...
	getting fielder columns...
selecting games and roster rows 1915 tot en met 1916...
	fixing pitch counts so they to be properly zero-padded when needed...
		CAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...
splitting play into play description and baserunning...
cleaning baserunning info...
creating a new dataframe to hold what we calculate here...


Read 3,111 games and 1,196 roster entries and 172,316 events in 14.9 seconds!


In [3]:
# games_df[
#     (games_df['visitor_linescore'].apply(lambda x: len(x) >= 9))
#     & (games_df['home_linescore'].apply(lambda x: len(x) >= 9))
# ][['visitor_linescore', 'home_linescore']]

#games_df.columns.tolist()[:]

#games_df
#games_df.loc['BOS191604120'].T[0:50]
# events_df[
#     (events_df['gameID'] == 'BLF191504100')
#     & (events_df['inning'] == 1)
# ][['event_in_game', 'inning', 'visitor_or_home', 'batterID', 'theplay']].reset_index().merge(
#     rosters_df[(rosters_df['theyear'] == 1915) & (rosters_df['team'].isin(['BLF', 'NEW']))
#               ], how='left', left_on='batterID', right_on='playerID').set_index('eventID')


In [4]:
start_at_game_number = 0

pai = 155

s = time.time()
# print('getting from backup...')
# games_df = games_df[games_df['gameno'].apply(lambda x: True == False)]
# rosters_df = rosters_df[rosters_df['theyear'].apply(lambda x: True == False)]
# events_df = events_df[events_df['event_in_game'].apply(lambda x: True == False)]
# games_df = games_df_bk
# rosters_df = rosters_df_bk
# events_df = events_df_bk


base_updater_atom_re = re.compile('(\d)+\((\d|B|H)\)')

fielder_out_re = re.compile('^\d+!*')

hit_re = re.compile('^(S|D|T|HR)(\d|GR)*$')
onbase_nohit_re = re.compile('^(\d|FL)*(W|IW|HP|C|E|FC)')

stolen_base_re = re.compile('SB(\d|H)')
caught_stealing_re = re.compile('CS(\d|H)')
pickoff_re = re.compile('PO\d\(*E*\d*\)')

error_re = re.compile('^(\d|FL)*E\d+')
baserunning_fielder_error_re = re.compile('(\d|B|H)\(\d*(E|PB)\d*\)')  # had to add the possibility of (NR)(*E)
nr_fixer_re = '\(NR\)'

print_these = {}
print_these['gamelog'] = False
print_these['events'] = True
print_these['baserunning'] = True
print_these['noplay'] = True

for x in print_these.keys():
    if (print_these[x]):
        print('\tprinting {0:} events...'.format(x))
    else:
        print('\tNOT printing {0:} events...'.format(x))

print('printing starting with the {0:} inning...'.format(ordinaltg(pai)))
print('\n')

killswitch = False
max_event_id = np.max(new_things_df.index.tolist())

courtesy_runner_half_inning = -1
score = [0,0]
onbase = {1: '', 2: '', 3: ''}

# NOTE: we HAVE TO keep all the NPs because pitches occurred on all of them!
print('\nparsing plays...')

# #gamelist = sorted(events_df['gameID'].unique().tolist())
gamelist = events_df['gameID'].unique().tolist()
print('Found {0:,.0f} events from {1:,.0f} games-with-events!'.format(len(events_df), len(gamelist)))
print('\n')

# nGames = len(gamelist)

for this_game_number in range(start_at_game_number, len(gamelist)):
    thisgameID = gamelist[this_game_number]

    if not(print_these['gamelog']):
        if (np.mod(this_game_number, 100) == 0):
            print('parsing game {0:.0f} ({1:}): {2:} at {3:} on {4:}...'.format(this_game_number, thisgameID, games_df.loc[thisgameID]['visitor'], games_df.loc[thisgameID]['home'], games_df.loc[thisgameID]['date'].strftime('%Y-%m-%d')))
    else:
        if (np.mod(this_game_number, 1) == 0):
            print('parsing game {0:.0f} ({1:}): {2:} at {3:} on {4:}...'.format(this_game_number, thisgameID, games_df.loc[thisgameID]['visitor'], games_df.loc[thisgameID]['home'], games_df.loc[thisgameID]['date'].strftime('%Y-%m-%d')))

    for i in [0,1]:
        score[i] = 0    
    this_game_innings = events_df[events_df['gameID'] == thisgameID]['inning'].unique()              
    for this_inning in this_game_innings:
        for vh in [0,1]:
            nOuts = 0
            for i in onbase:
                onbase[i] = ''    
            if (vh == 0):
                inningside = 'top'
            else:
                inningside = 'bottom'
            if ((print_these['gamelog']) & (this_inning >= pai)):
                print('{0:} of the {1:}...'.format(inningside.title(), ordinaltg(this_inning)))
            for ix, thisrow in events_df[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning) & (events_df['visitor_or_home'] == vh)].iterrows():
                new_things_df.loc[ix, ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b','runner_3b'
                                      ]] = [score[0], score[1], nOuts, onbase[1], onbase[2], onbase[3]]
                batter_event = True
                weird = False
                manual_batter_advance = False
                explicit_baserunning = False
                is_forceout = False
                
                if (type(thisrow['play_description']) == str):
                    if ('COUR' in thisrow['play_description']):
                        courtesy_runner_half_inning = vh

                # No play?
                if (thisrow['theplay'] == 'NP'):
                    if ((print_these['noplay']) & (this_inning >= pai)):
                        print('-------------- NO PLAY --------------')
                    explained = False
                    if (type(thisrow['comments']) == str):
                        if (print_these['noplay'] & (this_inning >= pai)):
                            print('\tEvent {0:0d} (ix={1:0d}): comment: {2:}'.format(thisrow['event_in_game'], ix, thisrow['comments']))
                            print('\n')
                        explained = True
                    if (type(thisrow['pinch_runner_playerID']) == str):
                        if (print_these['noplay'] & (this_inning >= pai)):
                            print('\tEvent {0:0d} (ix={1:0d}): pinch runner: {2:} replaces {3:}'.format(thisrow['event_in_game'], ix, thisrow['pinch_runner_playerID'], thisrow['pinch_runner_replaced_playerID']))
                            print('\n')
                        explained = True
                    if (thisrow['batterID'] != events_df.loc[ix+1]['batterID']):
                        if (print_these['noplay'] & (this_inning >= pai)):
                            print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at bat...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['batterID'], thisrow['batterID']))
                            print('\n')
                        explained = True
                    if (thisrow['pitcherID'] != events_df.loc[ix+1]['pitcherID']):
                        if (print_these['noplay'] & (this_inning >= pai)):
                            print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at pitcher...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['pitcherID'], thisrow['pitcherID']))
                            print('\n')
                        explained = True    
                    for this_fielder in fielder_columns:
                        if (thisrow[this_fielder] != events_df.loc[ix+1][this_fielder]):
                            if (print_these['noplay'] & (this_inning >= pai)):
                                print('\tEvent {0:0d} (ix={1:0d}): {2:} replaces {3:} at pitcher...'.format(thisrow['event_in_game'], ix, events_df.loc[ix+1]['pitcherID'], thisrow['pitcherID']))
                                print('\n')
                            explained = True
                    if (courtesy_runner_half_inning != vh):
                        explained = True
                        if (vh != courtesy_runner_half_inning):
                            courtesy_runner_half_inning = -1
                    if (not(explained)):
                        print('\tEvent {0:0d} (ix={1:0d}): UNEXPLAINED NO PLAY'.format(thisrow['event_in_game'], ix))
                        print('This row:')
                        print(thisrow)
                        print('\n')
                    if (print_these['noplay'] & (this_inning >= pai)):
                        print('-------------- /NO PLAY --------------')
                else:    # there is a play
                    if ((print_these['events']) & (this_inning >= pai)):
                        print('\tEvent {0:0d} (ix = {1:.0f}): {2:} at bat with {3:.0f} outs with score [V {4:0d} H {5:0d}] and onbase = {6:}...'.format(thisrow['event_in_game'], ix, thisrow['batterID'], nOuts, score[0], score[1], onbase))
                        print('\t\tPLAY: {0:} ({1:})...'.format(thisrow['theplay'], thisrow['play_orig']))
                    if ((re.search(base_updater_atom_re, thisrow['theplay']) != None) and ('CS' not in thisrow['theplay']) and ('PO' not in thisrow['theplay'])):
                        if (print_these['events'] & (this_inning >= pai)):
                            print('\t\t\tPlay found by base updater atom method in play: {0:} ({1:})...'.format(thisrow['theplay'], thisrow['play_orig']))
                        for this_base_updater_atom in re.finditer(base_updater_atom_re, thisrow['theplay']):
                            if (print_these['events'] & (this_inning >= pai)):
                                print('\t\t\t\t{0:}'.format(
                                    #thisrow['theplay'][0:this_base_updater_atom.start()], 
                                    thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]#,
                                    #thisrow['theplay'][this_base_updater_atom.end():]
                                ))
                            runner_from_out_str = thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()][re.search("\(",thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]).start()+1:re.search("\)",thisrow['theplay'][this_base_updater_atom.start():this_base_updater_atom.end()]).end()-1]
                            if (runner_from_out_str == 'B'):
                                if (print_these['events'] & (this_inning >= pai)):
                                    print('\t\t\t\tbatter out!')
                                manual_batter_advance = True
                                nOuts += 1
                            else:
                                runner_from_base_out = int(runner_from_out_str)
                                if (print_these['events'] & (this_inning >= pai)):
                                    print('\t\t\t\trunner coming from {0:} ({1:}) is out!'.format(ordinaltg(runner_from_base_out), onbase[runner_from_base_out]))
                                nOuts += 1
                                onbase[runner_from_base_out] = ''
                                if (not(manual_batter_advance)):
                                    if ('FO' in thisrow['play_description']):
                                        if (print_these['events'] & (this_inning >= pai)):
                                            print('\t\t\t\t\tThis is a forceout!')
                                        is_forceout = True
                        if ((print_these['events']) & (this_inning >= pai)):
                            print(re.findall(base_updater_atom_re, thisrow['theplay']))
                        #if ((len(re.findall(base_updater_atom_re, thisrow['theplay'])) > 1) & ('DP' in thisrow['play_description']) & ~(manual_batter_advance)):
                        if ((len(re.findall(base_updater_atom_re, thisrow['theplay'])) > 1) & ~(manual_batter_advance)):
                            is_forceout = True
                        if (print_these['events'] & (this_inning >= pai)):
                            print('{0:.0f} events long...'.format(len(re.findall(base_updater_atom_re, thisrow['theplay']))))
                            print('\n')
                    

                    #print('update baserunning for batter events...')
                    if (type(thisrow['baserunning_cleaned']) == str):
                        if (thisrow['theplay'][0:2] == 'HR'):
                            explicit_baserunning = True
                        if ((print_these['baserunning']) & (this_inning >= pai)):
                            print('parsing baserunning: {0:} <{1:}>...'.format(thisrow['baserunning_cleaned'], thisrow['play_orig']))
                        for this_advance in thisrow['baserunning_cleaned'].split(';'):
                            start_base = int(this_advance[0])
                            end_base = int(this_advance[2])
                            
                            if ((print_these['baserunning']) & (this_inning >= pai)):
                                print(this_advance, start_base, end_base)
                            
                            if (this_advance[1] == '-'):
                                if (end_base == 4):
                                    explicit_baserunning = True
                                    score[vh] += 1
                                    if (start_base == 0):
#                                         print('end base four start base zero')
#                                         print(score)
                                        manual_batter_advance = True
#                                        print(manual_batter_advance)
                                elif (start_base == 0):
                                    onbase[end_base] = thisrow['batterID']
                                    manual_batter_advance = True
                                else:
                                    onbase[end_base] = onbase[start_base]
                                if (start_base > 0):
                                    onbase[start_base] = ''
                            elif (this_advance[1] == 'X'):
#                                 print('-----------------------------')
#                                 print(this_advance)                            
                                #print(nr_fixer_re)
#                                 try:
#                                     print(this_advance[re.search(nr_fixer_re,this_advance).start():re.search(nr_fixer_re,this_advance).end()])
#                                 except:
#                                     print('Pattern not found:', re.search(nr_fixer_re, this_advance))
                                if (re.search(nr_fixer_re, this_advance)):
#                                    print('Pattern found!')
                                    nr_fixed_advance = this_advance[0:re.search(nr_fixer_re, this_advance).start()]+this_advance[re.search(nr_fixer_re, this_advance).end():]
                                else:
                                    nr_fixed_advance = this_advance
#                                 print(nr_fixed_advance)
#                                 print('-----------------------------')
#                                 print(baserunning_fielder_error_re)
#                                 print(re.search(baserunning_fielder_error_re, nr_fixed_advance))
                                if (re.search(baserunning_fielder_error_re, nr_fixed_advance) != None):
                                    if ((print_these['baserunning']) & (this_inning >= pai)):
                                        if (start_base == 0):
                                            failed_caught_runner = thisrow['batterID']
                                        else:
                                            failed_caught_runner = onbase[start_base]
                                        print('Failed caught stealing with NR: runner {0:} advances from {1:} to {2:}!'.format(failed_caught_runner, ordinaltg(start_base), ordinaltg(end_base)))
                                        print('---------------------------')
                                    if (end_base == 4):
                                        score[vh] += 1
                                    else:
#                                        print('Failed caught stealing WITHOUT NR: runner {0:} advances from {1:} to {2:}!'.format(failed_caught_runner, ordinaltg(start_base), ordinaltg(end_base)))
                                        if (start_base == 0):
                                            onbase[end_base] = thisrow['batterID']
                                            manual_batter_advance = True
                                        else:
                                            onbase[end_base] = onbase[start_base]
                                            onbase[start_base] = ''
                                else:
                                    if (end_base < 4):
                                        onbase[end_base] = ''
                                    nOuts += 1
                                    if (start_base == 0):
                                        manual_batter_advance = True
                                    else:
                                        onbase[start_base] = ''

                    if ('SB' in thisrow['theplay']):
#                        if (not(explicit_baserunning)):
                        if (print_these['events'] & (this_inning >= pai)):
                            print('onbase before: {0:}'.format(onbase))
                            print('stolen base!')
                            print(thisrow['theplay'])
                        for match in re.finditer(stolen_base_re, thisrow['theplay']):
                            if (thisrow['theplay'][match.start()+2:match.end()] == 'H'):
                                base_stolen = 4
                            else:
                                base_stolen = int(thisrow['theplay'][match.start()+2:match.end()])
                            if (print_these['events'] & (this_inning >= pai)):
                                print('\t\t\t\t{0:} base is stolen!'.format(ordinaltg(base_stolen)))
                            if (base_stolen == 4):
                                score[vh] += 1
                                for b in range(3, 0, -1):
                                    if (onbase[b] != ''):
                                        onbase[b] = ''
                                        break
                            else:
                                for b in range(base_stolen-1, 0, -1):
                                    if (onbase[b] != ''):
                                        onbase[base_stolen] = onbase[b]
                                        onbase[b] = ''
                                        break
                        if (print_these['events'] & (this_inning >= pai)):
                            print('onbase after: {0:}'.format(onbase))


                    if ('CS' in thisrow['theplay']):
                        if (print_these['events'] & (this_inning >= pai)):
                            print('caught stealing!')
                            print(thisrow['theplay'])
                        for match in re.finditer(caught_stealing_re, thisrow['theplay']):
                            if (thisrow['theplay'][match.start()+2:match.end()] == 'H'):
                                caught_at = 4
                            else:
                                caught_at = int(thisrow['theplay'][match.start()+2:match.end()])
                            if (re.search(baserunning_fielder_error_re, thisrow['theplay']) != None):
                                if (not(explicit_baserunning)):
                                    if ((print_these['events'] == True) & (this_inning >= pai)):
                                        print('\t\t\t\tfailed caught stealing, runner is safe at {0:}...'.format(caught_at))
                                    for b in range(caught_at-1,0,-1):
                                        if (onbase[b] != ''):
                                            start_base = b
                                            break
                                    if (caught_at == 4):
                                        score[vh] += 1
                                        if ((print_these['events'] == True) & (this_inning >= pai)):
                                            print('\t\t\t\t\trunner <{0:}> scores from {1:}, score now [V {2:0d} H {3:0d}]...'.format(onbase[start_base], start_base, score[0], score[1]))
                                    else:
                                        if ((print_these['events'] == True) & (this_inning >= pai)):
                                            print('\t\t\t\t\trunner <{0:}> advances from {1:} to {2:}...'.format(onbase[start_base], start_base, caught_at))
                            else:
                                if (caught_at == 4):
                                    pr = 'home'
                                else:
                                    pr = ordinaltg(caught_at)
                                for b in range(caught_at-1, 0, -1):
                                    if (onbase[b] != ''):
                                        caught_runner = onbase[b]
                                        onbase[b] = ''

                                        break
                                nOuts += 1
                                if (print_these['events'] & (this_inning >= pai)):
                                    print('runner caught stealing at {0:} for {1:} out...'.format(pr, ordinaltg(nOuts)))


                    if ('PO' in thisrow['theplay']):
                        if (print_these['events'] & (this_inning >= pai)):
                            print('pickoff!')
                            print(thisrow['theplay'])
                        for match in re.finditer(pickoff_re, thisrow['theplay']):
                            if (re.search(baserunning_fielder_error_re, thisrow['theplay']) != None):
                                if (print_these['events'] & (this_inning >= pai)):
                                    print('Failed pickoff: {0:}'.format(thisrow['theplay']))
                            else:
                                picked_off_at = int(thisrow['theplay'][match.start()+2:match.start()+3])
                                if (print_these['events'] & (this_inning >= pai)):
                                    print('picked off at {0:}... <{1:}>'.format(ordinaltg(picked_off_at), thisrow['theplay'][match.start():match.end()]))
                                onbase[picked_off_at] = ''
                                nOuts += 1
                    if (thisrow['theplay'][0] == 'K'):
                        if not(manual_batter_advance):
                            nOuts += 1
                    elif (((thisrow['theplay'][0] == 'W') & (thisrow['theplay'][0:2] != 'WP')) | (thisrow['theplay'][0:2] == 'HP') | (thisrow['theplay'][0:2] == 'IW') | ((thisrow['theplay'][0] == 'C') & (thisrow['theplay'][0:2] != 'CS'))):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif ((thisrow['theplay'][0] == 'S') & (thisrow['theplay'][0:2] != 'SB')):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'D'):
                        if not(manual_batter_advance):
                            onbase[2] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'T'):
                        if not(manual_batter_advance):
                            onbase[3] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] == 'HR'):
#                        print(onbase)
                        score[vh] += len([x for x in onbase.keys() if onbase[x] != ''])
#                         print(manual_batter_advance)
#                         print(score)
                        if (manual_batter_advance == False):
                            score[vh] += 1
#                        print(score)
                        for i in [1,2,3]:
                            onbase[i] = ''
                            
                    elif (re.search(error_re, thisrow['theplay']) != None):
                        if not(manual_batter_advance):
                            if (not(thisrow['theplay'][0:3] == 'FLE')):   # If the error was on a foul ball, the runner is not out
                                onbase[1] = thisrow['batterID']
                            if ((print_these['baserunning']) & (this_inning >= pai)):
                                print('ERROR ON PLAY, batter advances!')
                        
                    elif (re.search(fielder_out_re, thisrow['theplay'])):
                        if not(manual_batter_advance):
                            if (is_forceout):
                                onbase[1] = thisrow['batterID']
                            elif (nOuts < 3):
                                nOuts += 1
                    elif (thisrow['theplay'][0:2] == 'FC'):
                        if not(manual_batter_advance):
                            onbase[1] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] in ['SB', 'CS', 'PO', 'BK', 'OA', 'WP', 'PB']):
                        pass
                    else:
                        print('PLAY NOT FOUND!')
                        print('{0:} ({1:})'.format(thisrow['theplay'], thisrow['play_orig']))
                        killswitch = True
                    if ((print_these['events']) & (this_inning >= pai)):
                        print('\t\t\tOUTCOME: {0:} at bat with {1:.0f} outs with score [V {2:0d} H {3:0d}] and onbase = {4:}...'.format(thisrow['batterID'], nOuts, score[0], score[1], onbase))
                        print('\n')
#                     if ((thisrow['gameID'] == 'NY1191504290') and (thisrow['event_in_game'] == 3)):
#                         print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}!'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
#                         onbase[1] = ''
                    if ((thisrow['gameID'] == 'PHI191509092') and (thisrow['event_in_game'] == 40)):
                        print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
                        print('\t\tExplanation: fielding error but runner still out at 3rd')
                        onbase[3] = ''
                        nOuts += 1
                    if ((thisrow['gameID'] == 'PIT191606242') & (thisrow['event_in_game'] == 58)):
                        print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
                        print('\t\tExplanation: steal of home was counted in both baserunning and stolen base event')
                        score[1] = 4
                    if ((thisrow['gameID'] == 'PIT191607080') & (thisrow['event_in_game'] == 86)):
                        print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
                        print('\t\tExplanation: defensive indifference at 2nd should not advance batter')
                        onbase[2] = ''
                    if ((thisrow['gameID'] == 'SLN191606290') & (thisrow['event_in_game'] == 5)):
                        print('\tFixing a special case in game number {0:,.0f} ({1:}) at event in game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
                        print('\t\tExplanation: defensive indifference at 2nd should not advance batter')
                        score[0] = 1
                        

            # checks: each half-inning must end with 3 outs, and each inning must end with scores matching game log
            if (nOuts != 3):
                if (thisrow['gameID'] in early_end_game_list):
                    print('\tGAME NUMBER {0:,.0f} ENDED EARLY: {1:}'.format(this_game_number, thisrow['gameID']))
                else:
                    if not((this_inning == np.max(this_game_innings)) & (vh == 1)):
                        print('DISCREPANCY: {0:} of the {1:} has {2:0d} outs!'.format(inningside.title(), ordinaltg(this_inning), nOuts))
                        killswitch = True
                        max_event_id = ix
            skips = 0
            sco = 0
            if (games_df.loc[thisgameID]['has_linescores']):
                if (vh == 0):
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['visitor_linescore'][j] == '('):
                            visitor_linescore_raw = games_df.loc[thisgameID]['visitor_linescore']
                            skips = visitor_linescore_raw[j:].find(')')
                            sco += int(visitor_linescore_raw[visitor_linescore_raw.find('(')+1:visitor_linescore_raw.find(')')])
                        else:
                            sco += int(games_df.loc[thisgameID]['visitor_linescore'][j+skips])
                    if (score[0] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['visitor'], score[0], sco, games_df.loc[thisgameID]['visitor_linescore']))
                        killswitch = True
                        max_event_id = ix
                else:
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['home_linescore'][j+skips] != 'x'):
                            if (games_df.loc[thisgameID]['home_linescore'][j] == '('):
                                home_linescore_raw = games_df.loc[thisgameID]['home_linescore']
                                skips = home_linescore_raw[j:].find(')')
                                sco += int(home_linescore_raw[home_linescore_raw.find('(')+1:home_linescore_raw.find(')')])
                            else:
                                sco += int(games_df.loc[thisgameID]['home_linescore'][j+skips])
                    if (score[1] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['home'], score[1], sco, games_df.loc[thisgameID]['home_linescore']))
                        killswitch = True
                        max_event_id = ix
                #killswitch = False
    #             if ((print_these['gamelog']) & (this_inning >= pai)):
    #                 print('\n')
            
            if (killswitch):
                break
        if (killswitch):
            break
    if (killswitch):
        break

events_contexts_df = pandas.concat((events_df.loc[events_df.index <= max_event_id], new_things_df.loc[new_things_df.index <= max_event_id]), axis=1)
e = time.time()
g += e-s
print('\n')
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(events_contexts_df), np.floor((e-s)/60), (e-s)%60))



	NOT printing gamelog events...
	printing events events...
	printing baserunning events...
	printing noplay events...
printing starting with the 155th inning...



parsing plays...
Found 172,316 events from 2,080 games-with-events!


parsing game 0 (BLF191504100): NEW at BLF on 1915-04-10...
parsing game 100 (BRF191508071): CHF at BRF on 1915-08-07...
parsing game 200 (CHF191506060): PTF at CHF on 1915-06-06...
parsing game 300 (NEW191506290): CHF at NEW on 1915-06-29...
parsing game 400 (SLF191505311): PTF at SLF on 1915-05-31...
parsing game 500 (CHA191507190): BOS at CHA on 1915-07-19...
parsing game 600 (NYA191506150): SLA at NYA on 1915-06-15...
parsing game 700 (PHA191508100): CHA at PHA on 1915-08-10...
parsing game 800 (BRO191507071): BSN at BRO on 1915-07-07...
parsing game 900 (BSN191509020): BRO at BSN on 1915-09-02...
parsing game 1000 (CIN191509140): NY1 at CIN on 1915-09-14...
parsing game 1100 (PHI191505200): CIN at PHI on 1915-05-20...
	GAME NUMBER 1,125 ENDED EARLY: PH

In [5]:
s = time.time()
events_contexts_df = pandas.concat((events_df, new_things_df), axis=1)
#new_things_df
events_contexts_df.columns.tolist()

events_contexts_df = events_contexts_df[final_column_order]
events_contexts_df.to_csv('events_with_context_{0:0d}_{1:0d}.csv'.format(firstyear, finalyear))
e = time.time()
g += e-s
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(new_things_df), np.floor(g/60), g%60))


Processed 172,316 events in 13 minutes 41 seconds!


In [6]:
#games_df.loc['BLF191504100'][['visitor_linescore', 'home_linescore', 'has_linescores']]#.groupby('has_linescores').size()